Code doesn't drop latitude or longitude
https://www.kaggle.com/datasets/sadeghjalalian/ufo-sightings-in-usa

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import numpy as np
import tensorflow as tf
from pathlib import Path
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
#  Import and read the charity_data.csv.
ufo = pd.read_csv("./Resources/nuforc_reports.csv")
ufo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136940 entries, 0 to 136939
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   summary         136866 non-null  object 
 1   city            136562 non-null  object 
 2   state           127595 non-null  object 
 3   date_time       134272 non-null  object 
 4   shape           131018 non-null  object 
 5   duration        130451 non-null  object 
 6   stats           136940 non-null  object 
 7   report_link     136940 non-null  object 
 8   text            136902 non-null  object 
 9   posted          134272 non-null  object 
 10  city_latitude   110136 non-null  float64
 11  city_longitude  110136 non-null  float64
dtypes: float64(2), object(10)
memory usage: 12.5+ MB


In [3]:
ufo.head()

,summary,city,state,date_time,shape,duration,stats,report_link,text,posted,city_latitude,city_longitude
0,Viewed some red lights in the sky appearing to...,Visalia,CA,2021-12-15T21:45:00,light,2 minutes,Occurred : 12/15/2021 21:45 (Entered as : 12/...,http://www.nuforc.org/webreports/165/S165881.html,Viewed some red lights in the sky appearing to...,2021-12-19T00:00:00,36.356650,-119.347937
1,Look like 1 or 3 crafts from North traveling s...,Cincinnati,OH,2021-12-16T09:45:00,triangle,14 seconds,Occurred : 12/16/2021 09:45 (Entered as : 12/...,http://www.nuforc.org/webreports/165/S165888.html,Look like 1 or 3 crafts from North traveling s...,2021-12-19T00:00:00,39.174503,-84.481363
2,seen dark rectangle moving slowly thru the sky...,Tecopa,CA,2021-12-10T00:00:00,rectangle,Several minutes,Occurred : 12/10/2021 00:00 (Entered as : 12/...,http://www.nuforc.org/webreports/165/S165810.html,seen dark rectangle moving slowly thru the sky...,2021-12-19T00:00:00,NaN,NaN
3,"One red light moving switly west to east, beco...",Knoxville,TN,2021-12-10T19:30:00,triangle,20-30 seconds,Occurred : 12/10/2021 19:30 (Entered as : 12/...,http://www.nuforc.org/webreports/165/S165825.html,"One red light moving switly west to east, beco...",2021-12-19T00:00:00,35.961561,-83.980115
4,"Bright, circular Fresnel-lens shaped light sev...",Alexandria,VA,2021-12-07T08:00:00,circle,NaN,Occurred : 12/7/2021 08:00 (Entered as : 12/0...,http://www.nuforc.org/webreports/165/S165754.html,"Bright, circular Fresnel-lens shaped light sev...",2021-12-19T00:00:00,38.798958,-77.095133


In [4]:
#Convert to dataframe
ufo_df = ufo
#look at State value counts for binning
ufo_count = ufo_df.state.value_counts()
ufo_count

CA                              15124
FL                               7551
WA                               6744
TX                               5650
NY                               5424
                                ...  
QUEBEC                              1
UNITED STATES VIRGIN ISLANDS        1
TAIPEI (TAIWAN)                     1
VICTORIA                            1
UNITED KINGDOM                      1
Name: state, Length: 86, dtype: int64

In [5]:
# Columns
ufo_df.columns

Index(['summary', 'city', 'state', 'date_time', 'shape', 'duration', 'stats',
       'report_link', 'text', 'posted', 'city_latitude', 'city_longitude'],
      dtype='object')

In [6]:
# List dataframe data types
ufo_df.dtypes

summary            object
city               object
state              object
date_time          object
shape              object
duration           object
stats              object
report_link        object
text               object
posted             object
city_latitude     float64
city_longitude    float64
dtype: object

In [7]:
newufo_df = ufo_df[(ufo_df.state == "AZ")]
newufo_df

,summary,city,state,date_time,shape,duration,stats,report_link,text,posted,city_latitude,city_longitude
60,"When we were about 17-18, a friend and I had j...",Encinitas,AZ,1973-11-03T15:00:00,disk,10 seconds,Occurred : 11/3/1973 15:00 (Entered as : 11/0...,http://www.nuforc.org/webreports/154/S154888.html,"When we were about 17-18, a friend and I had j...",2020-04-09T00:00:00,NaN,NaN
117,Fleet of moving stars. ((Starlink satellites?)),Litchfield Park,AZ,2020-04-17T20:00:00,light,1 hour,Occurred : 4/17/2020 20:00 (Entered as : 04/1...,http://www.nuforc.org/webreports/155/S155338.html,Fleet of moving stars Looked like moving stars...,2020-06-25T00:00:00,33.517800,-112.366700
125,Crazy light show! ((Starlink satellites?)),Tucson,AZ,2020-04-15T20:30:00,light,20 minutes,Occurred : 4/15/2020 20:30 (Entered as : 4/15...,http://www.nuforc.org/webreports/155/S155177.html,Crazy light show! A series of lights eminating...,2020-06-25T00:00:00,32.259941,-110.927542
134,Star looking ufo in sky starts to move then di...,Phoenix,AZ,2020-04-12T01:00:00,other,5-7 minutes,Occurred : 4/12/2020 01:00 (Entered as : 04/1...,http://www.nuforc.org/webreports/154/S154998.html,Star looking ufo in sky starts to move then di...,2020-06-25T00:00:00,33.535381,-112.049460
145,"Saw a bright, star like object, very high up, ...",Tucson,AZ,2020-03-29T21:00:00,circle,2 hours,Occurred : 3/29/2020 21:00 (Entered as : 03/2...,http://www.nuforc.org/webreports/154/S154641.html,"Saw a bright, star like object, very high up, ...",2020-06-25T00:00:00,32.259941,-110.927542
...,...,...,...,...,...,...,...,...,...,...,...,...
136848,Tonopah Arizona lights,Tonopah,AZ,2020-07-23T19:30:00,fireball,1 minute,Occurred : 7/23/2020 19:30 (Entered as : 07/2...,http://www.nuforc.org/webreports/158/S158188.html,Tonopah Arizona lights It was in the afternoon...,2020-08-06T00:00:00,33.577800,-113.023400
136866,MADAR Node 74,Show Low,AZ,2020-06-12T13:46:00,NaN,NaN,Occurred : 6/12/2020 13:46 (Entered as : 06/1...,http://www.nuforc.org/webreports/157/S157391.html,MADAR Node 74,2020-07-09T00:00:00,34.234662,-110.075197
136883,Tonopah Arizona lights,Tonopah,AZ,2020-07-23T19:30:00,fireball,1 minute,Occurred : 7/23/2020 19:30 (Entered as : 07/2...,http://www.nuforc.org/webreports/157/S157699.html,Tonopah Arizona lights It was in the afternoon...,2020-07-23T00:00:00,33.577800,-113.023400
136897,It was about 2:15am to 2:30am I look up in the...,Scottsdale,AZ,2019-07-08T02:15:00,circle,15 min,Occurred : 7/8/2019 02:15 (Entered as : 07/08...,http://www.nuforc.org/webreports/147/S147747.html,It was about 2:15am to 2:30am I look up in the...,2019-07-25T00:00:00,33.564138,-111.905083


In [8]:
#look at State value counts for binning
newufo_count = newufo_df.state.value_counts()
newufo_count

AZ    4625
Name: state, dtype: int64

In [9]:
# Find null values
for column in newufo_df.columns:
    print(f"Column {column} has {newufo_df[column].isnull().sum()} null values")

Column summary has 2 null values
Column city has 7 null values
Column state has 0 null values
Column date_time has 25 null values
Column shape has 176 null values
Column duration has 203 null values
Column stats has 0 null values
Column report_link has 0 null values
Column text has 2 null values
Column posted has 25 null values
Column city_latitude has 619 null values
Column city_longitude has 619 null values


In [ ]:
# drop null rows  latitude and longitude only which are float64   NaN
#newufo_df.dropna(subset=newufo_df.select_dtypes(include=np.number).columns, how='all')
#newufo_df1 = newufo_df.dropna(subset=["city_latitude","city_longitude"])

In [18]:
# find duplicate entries
print(f"Duplicate entries: {newufo_df.duplicated().sum()}")

Duplicate entries: 0


AZ    4625
Name: state, dtype: int64

Column summary has 2 null values
Column city has 7 null values
Column state has 0 null values
Column date_time has 25 null values
Column shape has 176 null values
Column duration has 203 null values
Column stats has 0 null values
Column report_link has 0 null values
Column text has 2 null values
Column posted has 25 null values
Column city_latitude has 619 null values
Column city_longitude has 619 null values


In [19]:
# Remove columns not needed
newufo_df.drop(columns=["report_link"], inplace=True)
newufo_df.head()

,summary,city,state,date_time,shape,duration,stats,text,posted,city_latitude,city_longitude
60,"When we were about 17-18, a friend and I had j...",Encinitas,AZ,1973-11-03T15:00:00,disk,10 seconds,Occurred : 11/3/1973 15:00 (Entered as : 11/0...,"When we were about 17-18, a friend and I had j...",2020-04-09T00:00:00,NaN,NaN
117,Fleet of moving stars. ((Starlink satellites?)),Litchfield Park,AZ,2020-04-17T20:00:00,light,1 hour,Occurred : 4/17/2020 20:00 (Entered as : 04/1...,Fleet of moving stars Looked like moving stars...,2020-06-25T00:00:00,33.517800,-112.366700
125,Crazy light show! ((Starlink satellites?)),Tucson,AZ,2020-04-15T20:30:00,light,20 minutes,Occurred : 4/15/2020 20:30 (Entered as : 4/15...,Crazy light show! A series of lights eminating...,2020-06-25T00:00:00,32.259941,-110.927542
134,Star looking ufo in sky starts to move then di...,Phoenix,AZ,2020-04-12T01:00:00,other,5-7 minutes,Occurred : 4/12/2020 01:00 (Entered as : 04/1...,Star looking ufo in sky starts to move then di...,2020-06-25T00:00:00,33.535381,-112.049460
145,"Saw a bright, star like object, very high up, ...",Tucson,AZ,2020-03-29T21:00:00,circle,2 hours,Occurred : 3/29/2020 21:00 (Entered as : 03/2...,"Saw a bright, star like object, very high up, ...",2020-06-25T00:00:00,32.259941,-110.927542


In [15]:
# convert date to datestri
newufo_df["date_time"]

60        1973-11-03T15:00:00
117       2020-04-17T20:00:00
125       2020-04-15T20:30:00
134       2020-04-12T01:00:00
145       2020-03-29T21:00:00
                 ...         
136848    2020-07-23T19:30:00
136866    2020-06-12T13:46:00
136883    2020-07-23T19:30:00
136897    2019-07-08T02:15:00
136909    2019-04-01T23:00:00
Name: date_time, Length: 4625, dtype: object

In [20]:
newufo_df[newcolumnlist] = newufo_df["date_time"].str.split("T", expand=True)

NameError: name 'newufo_df1' is not defined

In [ ]:
newufo_df1


In [ ]:
# converting to csv
newufo_df1.to_csv('Resources/ufo_data_cleaned.csv')